<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Contexto Laboratorio 2</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2025-1</h2>
</center>

El objetivo del Laboratorio 2 es evaluar los contenidos de **Sistemas de Ecuaciones**, por lo que cualquiera o todos los métodos vistos en clases pueden ser requeridos.

In [2]:

import numpy as np

*Las siguientes implementaciones de los métodos se encuentran en el repositorio en GitHub de los Jupyter Notebooks del curso, link disponible en AULA*.

In [4]:
def solve_triangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform backward-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x


#permutation between rows i and j on matrix A
def row_perm(A, i, j):
    tmp = np.copy(A[i])
    A[i] = A[j]
    A[j] = tmp


def palu_decomp(A, show=False, print_precision=2):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    if show:
        print('Initial matrices')
        print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
        print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
        print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
        print('----------------------------------------')
    #iterating through columns
    for j in range(N-1):
        #determine the new pivot
        p_index = np.argmax(np.abs(U[j:,j]))
        if p_index != 0:
            row_perm(P, j, j+p_index)
            row_perm(U, j, j+p_index)
            row_perm(L, j, j+p_index)
            if show:
                print('A permutation has been made')
                print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
                print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
                print('----------------------------------------')
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j]
            if show:
                print('P = '); print(np.array_str(P, precision=print_precision, suppress_small=True))
                print('L = '); print(np.array_str(L+np.eye(N), precision=print_precision, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=print_precision, suppress_small=True))
                print('----------------------------------------')
    np.fill_diagonal(L,1)
    return P,L,U


def solve_palu(A, b, show=False, print_precision=2):
    P,L,U = palu_decomp(A, show, print_precision=print_precision)
    # A.x = b -> P.A.x = P.b = b' -> L.U.x = b'
    b = np.dot(P,b)
    # L.c = b' with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    return x


def jacobi(A, b, n_iter=50):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    #initial guess
    X[0] = np.zeros(n)
    #submatrices
    D = np.diag(A)
    Dinv = D**-1
    R = A - np.diag(D) # R = (L+U)
    for i in range(1, n_iter):
        # X[i] = Dinv*(b - np.dot(R, X[i-1]))
        # v1.12
        ri = b - np.dot(A, X[i-1])
        X[i] = X[i-1]+Dinv*ri # = np.dot(np.linalg.inv(D),ri)
    return X

In [10]:
A_prim = np.array([[3., -1., -.5, -1.],
                 [1., -3., -.25, 0.],
                 [.5, .25, 3., -.5],
                 [1., 0., 1., -3.]])

row_perm(A_prim, 0, 1)
print("A:\n", A_prim)
palu_a = solve_palu(A_prim, np.array([-5, 0, 0, 5]))
jacobi_a = jacobi(A_prim, np.array([-5, 0, 0, 5]))[-1]
print("Palu solution:\n", palu_a)
print("Jacobi solution:\n", jacobi_a)
row_perm(A_prim, 0, 1)
print("A:\n", A_prim)
palu_b = solve_palu(A_prim, np.array([-5, 0, 0, 5]))
jacobi_b = jacobi(A_prim, np.array([-5, 0, 0, 5]))[-1]
print("Palu solution:\n", palu_b)
print("Jacobi solution:\n", jacobi_b)

A:
 [[ 1.   -3.   -0.25  0.  ]
 [ 3.   -1.   -0.5  -1.  ]
 [ 0.5   0.25  3.   -0.5 ]
 [ 1.    0.    1.   -3.  ]]
Palu solution:
 [-0.13636364  1.65656566 -0.42424242 -1.85353535]
Jacobi solution:
 [-4.00140747e+23  1.34373671e+23  1.89333100e+21  1.58269187e+22]
A:
 [[ 3.   -1.   -0.5  -1.  ]
 [ 1.   -3.   -0.25  0.  ]
 [ 0.5   0.25  3.   -0.5 ]
 [ 1.    0.    1.   -3.  ]]
Palu solution:
 [-2.81818182 -0.94949495  0.12121212 -2.56565657]
Jacobi solution:
 [-2.81818182 -0.94949495  0.12121212 -2.56565657]


# Contexto

En una red de canales hidráulicos, el flujo de agua entre nodos (conexiones entre embalses, estaciones o compuertas) depende de las diferencias de presión y de la resistencia de cada canal (por fricción, estrechamiento, etc.). Supondremos una red plana con nodos conectados entre sí, donde el flujo cumple una versión discreta de la **ley de conservación de masa** (la suma de los flujos entrantes menos los salientes es igual a cero).

Dada una red de canales representada por un **grafo dirigido**, donde cada arista tiene una resistencia hidráulica $R_{ij}$, se requiere encontrar las presiones $p_{1}, p_{2}, \dots, p_{n}$ en los nodos, tal que se conserve el flujo y se respeten los caudales conocidos.

Cada canal (arista del grafo) entre los nodos $i$ y $j$ tiene un flujo que depende de la diferencia de presión entre sus extremos y su propia resistencia hidráulica:

$$
q_{ij} = \frac{p_{i} - p_{j}}{R_{ij}}
$$

La **conservación de masa** para el nodo $i$ implica que para cada nodo:

$$
\sum_{j \in \text{vecinos}(i)} q_{ij} = \sum_{\text{entradas}(i)} \frac{p_{j} - p_{i}}{R_{ji}} + \sum_{\text{salidas}(i)} \frac{p_{i} - p_{j}}{R_{ij}} = d_{i}
$$

donde $d_{i}$ es el **caudal neto** en el nodo $i$ (positivo si entra agua, negativo si sale).

Considere el siguiente grafo dirigido que representa una red de canales hidráulicos:

<p align="center">
  <img src="img/red.png" alt="Red de canales hidráulicos" style="margin:auto;width:500px;"/>
</p>

Para esta red, el desarrollo de la conservación de masa en cada nodo entrega el siguiente sistema de ecuaciones:

$$
\begin{align*}
(\text{Nodo } 1) \quad & \frac{p_{1} - p_{2}}{R_{12}} + \frac{p_{1} - p_{3}}{R_{13}} + \frac{p_{1} - p_{4}}{R_{14}} = -5 \\
(\text{Nodo } 2) \quad & \frac{p_{1} - p_{2}}{R_{12}} + \frac{p_{2} - p_{3}}{R_{23}} = 0 \\
(\text{Nodo } 3) \quad & \frac{p_{2} - p_{3}}{R_{23}} + \frac{p_{1} - p_{3}}{R_{13}} + \frac{p_{3} - p_{4}}{R_{34}} = 0 \\
(\text{Nodo } 4) \quad & \frac{p_{1} - p_{4}}{R_{14}} + \frac{p_{3} - p_{4}}{R_{34}} = 5
\end{align*}
$$

Al factorizar estas ecuaciones, obtenemos:

$$
\begin{align*}
(\text{Nodo } 1) \quad & p_{1} \left(\frac{1}{R_{12}} + \frac{1}{R_{13}} + \frac{1}{R_{14}}\right) + p_{2} \left(-\frac{1}{R_{12}}\right) + p_{3} \left(-\frac{1}{R_{13}}\right) + p_{4} \left(-\frac{1}{R_{14}}\right) = -5 \\
(\text{Nodo } 2) \quad & p_{1} \left(\frac{1}{R_{12}}\right) + p_{2} \left(\frac{1}{R_{23}} - \frac{1}{R_{12}}\right) + p_{3} \left(-\frac{1}{R_{23}}\right) = 0 \\
(\text{Nodo } 3) \quad & p_{1} \left(\frac{1}{R_{13}}\right) + p_{2} \left(\frac{1}{R_{23}}\right) + p_{3} \left(-\frac{1}{R_{23}} - \frac{1}{R_{13}} - \frac{1}{R_{34}}\right) + p_{4} \left(-\frac{1}{R_{34}}\right) = 0 \\
(\text{Nodo } 4) \quad & p_{1} \left(\frac{1}{R_{14}}\right) + p_{3} \left(\frac{1}{R_{34}}\right) + p_{4} \left(-\frac{1}{R_{14}} - \frac{1}{R_{34}}\right) = 5
\end{align*}
$$

Esto permite expresar el sistema en forma matricial $A\vec{x} = \vec{b}$:

$$
\begin{bmatrix}
\frac{1}{R_{12}} + \frac{1}{R_{13}} + \frac{1}{R_{14}} & -\frac{1}{R_{12}} & -\frac{1}{R_{13}} & -\frac{1}{R_{14}} \\
\frac{1}{R_{12}} & \frac{1}{R_{23}} - \frac{1}{R_{12}} & -\frac{1}{R_{23}} & 0 \\
\frac{1}{R_{13}} & \frac{1}{R_{23}} & -\frac{1}{R_{23}} - \frac{1}{R_{13}} - \frac{1}{R_{34}} & -\frac{1}{R_{34}} \\
\frac{1}{R_{14}} & 0 & \frac{1}{R_{34}} & -\frac{1}{R_{14}} - \frac{1}{R_{34}}
\end{bmatrix}
\begin{bmatrix}
p_{1} \\
p_{2} \\
p_{3} \\
p_{4}
\end{bmatrix}
=
\begin{bmatrix}
-5 \\
0 \\
0 \\
5
\end{bmatrix}
$$

Este es un sistema lineal de $4 \times 4$ que puede resolverse con métodos numéricos como **PALU** (descomposición LU con pivoteo) o **Jacobi**. Para ello, se asignarán los valores de resistencia:  
$R_{12} = 1$, $R_{13} = 2$, $R_{14} = 1$, $R_{23} = 4$, $R_{34} = 1$.

In [3]:
A = np.array([[2.5, -1., -.5, -1.],
              [1., -.75, -.25, 0.],
              [.5, .25, 0.25, -.5],
              [1., 0., 1., -2.]])

b = np.array([-5, 0, 0, 5])

print("A:\n",A)
print("b:\n",b)

A:
 [[ 2.5  -1.   -0.5  -1.  ]
 [ 1.   -0.75 -0.25  0.  ]
 [ 0.5   0.25  0.25 -0.5 ]
 [ 1.    0.    1.   -2.  ]]
b:
 [-5  0  0  5]


In [4]:
x_palu = solve_palu(A,b)
print("PALU; x:",x_palu)

PALU; x: [-0.625 -4.375 10.625  2.5  ]


In [5]:
x_jacobi = jacobi(A,b)[-1]
print("JACOBI; x:",x_jacobi)

JACOBI; x: [11.35856885 13.33549134 -1.25947294 10.88728433]


In [6]:
error = np.linalg.norm(x_palu-x_jacobi)
print("Error:",error)

Error: 25.86222460380605


### Comparación entre PALU y Jacobi para el sistema hidráulico

En este caso, el método de Jacobi no entregó el mismo resultado que PALU, lo cual se debe a que la matriz $A$ no es diagonal dominante (condición necesaria para Jacobi). Sin embargo, si por ejemplo la matriz $A$ hubiese resultado en:

$$
A' =
\begin{bmatrix}
3 & -1 & -0.5 & -1 \\
1 & -3 & -0.25 & 0 \\
0.5 & 0.25 & 3 & -0.5 \\
1 & 0 & 1 & -3 \\
\end{bmatrix}
$$

la cual sí es diagonal dominante, el método de Jacobi converge correctamente a la solución, utilizando menos recursos que PALU, especialmente en instancias grandes. A continuación se resuelve el sistema con $A'$.

In [7]:
A_prim = np.array([[3., -1., -.5, -1.],
                 [1., -3., -.25, 0.],
                 [.5, .25, 3., -.5],
                 [1., 0., 1., -3.]])
print("A´:\n",A_prim)
print("b:\n",b)

A´:
 [[ 3.   -1.   -0.5  -1.  ]
 [ 1.   -3.   -0.25  0.  ]
 [ 0.5   0.25  3.   -0.5 ]
 [ 1.    0.    1.   -3.  ]]
b:
 [-5  0  0  5]


In [8]:
x_prim_palu = solve_palu(A_prim,b)
print("PALU; x´:",x_prim_palu)

PALU; x´: [-2.81818182 -0.94949495  0.12121212 -2.56565657]


In [9]:
x_prim_jacobi = jacobi(A_prim,b)[-1]
print("JACOBI; x´:",x_prim_jacobi)

JACOBI; x´: [-2.81818182 -0.94949495  0.12121212 -2.56565657]


In [10]:
error_prim = np.linalg.norm(x_prim_palu-x_prim_jacobi)
print("Error:",error_prim)

Error: 1.1102230246251565e-16


Un problema análogo en el cual se puede realizar este análisis es al determinar las corrientes involucradas en un circuito eléctrico donde se conocen las conexiones y los voltajes, aplicando las leyes de Kirchhoff.

Las leyes de Kirchhoff se basan en el principio de conservación de carga y energía, pero antes de estudiarlas, es necesario hacer algunas definiciones:

* **Nodo**: Intersección de tres o más conductores.
* **Malla**: Todo recorrido cerrado en un circuito.
* **Rama**: Elemento o grupo de elementos conectados entre dos nodos.

### Leyes fundamentales

1. **Ley de Corrientes de Kirchhoff (LCK)**:
   Basada en la conservación de carga en cada nodo:
   $$ \sum_{k=1}^{n} I_k = 0 $$

2. **Ley de Voltajes de Kirchhoff (LVK)**:
   Establece la conservación de energía en mallas:
   $$ \sum_{k=1}^{n} V_k = 0 $$

3. **Ley de Ohm**:
   Relación fundamental entre voltaje, corriente y resistencia:
   $$ V_i = I_i \cdot R_i $$

### Análisis del circuito eléctrico

Considere el siguiente circuito:
<p align="center">
  <img src="img/circuito.png" alt="Circuito eléctrico" style="margin:auto;width:500px;"/>
</p>

#### Ecuaciones derivadas:

1. **Aplicando LVK a las mallas**:
   $$ \begin{align*}
   (Malla\;1) & \quad V_1 + V_3 = 24 \\
   (Malla\;2) & \quad V_2 - V_3 = -2
   \end{align*} $$

2. **Aplicando LCK al nodo superior**:
   $$ I_1 = I_2 + I_3 $$

Aplicando la ley de Ohm y reordenando términos se puede llegar al siguiente sistema en forma matricial $A\vec{x} = \vec{b}$:  

$$
\left[\begin{array}{ccc}
R_1 & 0 & R_3 \\
0 & -R_2 & R_3 \\
1 & -1 & -1
\end{array}\right]\left[\begin{array}{l}
I_1 \\
I_2 \\
I_3
\end{array}\right]=\left[\begin{array}{c}
24 \\
2 \\
0
\end{array}\right]
$$

Este es un sistema lineal de $3 \times 3$ que puede resolverse con métodos numéricos como **PALU** (descomposición LU con pivoteo) o **Jacobi**. Para ello:

In [11]:
A = np.array([
    [1.,  0.,  1.],
    [0., -2.,  1.],
    [1., -1., -1.]])

b = np.array([24., 2., 0.])

print("A:\n",A)
print("b:\n",b)

A:
 [[ 1.  0.  1.]
 [ 0. -2.  1.]
 [ 1. -1. -1.]]
b:
 [24.  2.  0.]


In [12]:
x_palu = solve_palu(A,b)
print("PALU; x:",x_palu)

PALU; x: [14.  4. 10.]


In [13]:
x_jacobi = jacobi(A,b)[-1]
print("JACOBI; x:",x_jacobi)

JACOBI; x: [ 168355.12196028  -84166.56098014 -168331.12196028]


In [14]:
error = np.linalg.norm(x_palu-x_jacobi)
print("Error:",error)

Error: 252511.6829404235


### Comparación entre PALU y Jacobi para el circuito

Nuevamente el método de jacobi no entregó el mismo resultado que PALU, debido a a que la matríz A no es diagonal dominante, si por ejemplo la matríz A hubiese resultado en:
$$ A^{'} =
\left[\begin{array}{ccc}
2 & 0 & 1 \\
0 & -2 & 1 \\
1 & -1 & -4
\end{array}\right]
$$
La cual sí es diagonal dominante, el método de Jacobi converge correctamente a la solución. A continuación se resuelve el sistema con $A^{'}$.

In [15]:
A_prim = np.array([
    [2.,  0.,  1.],
    [0., -2.,  1.],
    [1., -1., -4.]])
print("A´:\n",A_prim)
print("b:\n",b)

A´:
 [[ 2.  0.  1.]
 [ 0. -2.  1.]
 [ 1. -1. -4.]]
b:
 [24.  2.  0.]


In [16]:
x_prim_palu = solve_palu(A_prim,b)
print("PALU; x´:",x_prim_palu)

PALU; x´: [10.7  0.3  2.6]


In [17]:
x_prim_jacobi = jacobi(A_prim,b)[-1]
print("JACOBI; x´:",x_prim_jacobi)

JACOBI; x´: [10.7  0.3  2.6]


In [18]:
error_prim = np.linalg.norm(x_prim_palu-x_prim_jacobi)
print("Error:",error_prim)

Error: 1.1709563872695136e-14
